In [2]:
import pandas as pd
import numpy as np
from PIL import Image
import cv2

import os
# import random
# import shutil

In [14]:
data_dir = os.getcwd() + "/datasets"
train_dir = data_dir + "/train"
val_dir = data_dir + "/val"
test_dir = data_dir +  "/test"

#### Loading each image and their label into an array

In [15]:
def preprocessing(img):
    # common size for images
    IMG_HEIGHT, IMG_WIDTH = 128, 128
    img_array = cv2.imread(img)
    img_resized = cv2.resize(img_array, (IMG_WIDTH, IMG_HEIGHT))
    img_norm = img_resized
    return img_norm

X = [] # images
y = [] # labels (0 for benign, 1 for malignant)
splits = ['train', 'val', 'test']

def create_arrays(split):
    X = []
    y = []
    if split == 'train':
        img_dir = train_dir
    if split == 'val':
        img_dir = val_dir
    if split == 'test':
        img_dir = test_dir

    classes = ['benign', 'malignant']

    for class_name in classes:
        
        class_dir = os.path.join(img_dir, class_name)
        
        images = [f for f in os.listdir(class_dir) if not f.startswith('.DS_Store')]

        for img in images:
            img = preprocessing(os.path.join(class_dir, img))
            X.append(img)
            if class_name == 'benign':
                y.append(0)
            if class_name == 'malignant':
                y.append(1)

    X = np.array(X)
    y = pd.DataFrame(y, columns=['class_name'])

    return X, y

X_train, y_train = create_arrays('train')
X_val, y_val = create_arrays('val')
X_test, y_test = create_arrays('test')

In [16]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(396, 128, 128, 3)
(108, 128, 128, 3)
(43, 128, 128, 3)


#### Normalize image data

In [18]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

for image in X_train:
    for channel in range(image.shape[2]):
        image[:, :, channel] = ss.fit_transform(image[:, :, channel])
for image in X_val:
    for channel in range(image.shape[2]):
        image[:, :, channel] = ss.transform(image[:, :, channel])
for image in X_test:
    for channel in range(image.shape[2]):
        image[:, :, channel] = ss.transform(image[:, :, channel])    

#### Handle class imbalance

In [33]:
from sklearn.utils import class_weight

y_labels = y_train['class_name'].values
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_labels), y=y_labels)
class_weights = dict(enumerate(class_weights))
print(class_weights)

{0: 0.7071428571428572, 1: 1.706896551724138}


#### Model Building (Part 1)

In [39]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D


In [40]:
num_classes = 1

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

/Users/jenniferjordache/Developer/Kaggle-Breast-Cancer/breast_cancer_venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
''' Code for splitting images into validation set

validation_split = 0.2  # 20%

classes = ['benign', 'malignant']

for class_name in classes:
    class_train_dir = os.path.join(train_dir, class_name)
    class_val_dir = os.path.join(val_dir, class_name)

    image_files = [f for f in os.listdir(class_train_dir) if os.path.isfile(os.path.join(class_train_dir, f))]

    num_validation_images = int(len(image_files) * validation_split)

    validation_images = random.sample(image_files, num_validation_images)

    for img in validation_images:
        src = os.path.join(class_train_dir, img)
        dst = os.path.join(class_val_dir, img)
        shutil.move(src, dst)

'''